In [5]:
from gensim_preprocessing import corpus, dictionary, texts
import numpy as np
import json
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [6]:
bigrams_phrases = gensim.models.Phrases(texts, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[texts], threshold=100)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(textss):
    return [bigram[doc] for doc in textss]

def make_trigrams(textss):
    return [trigram[bigram[doc]] for doc in textss]

data_bigrams = make_bigrams(texts)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print(data_bigrams_trigrams[0])


NameError: name 'bigram_phrases' is not defined

In [ ]:
# TF-IDF REMOVAL

In [ ]:
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts_removal = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts_removal]
print(corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word )

low_value = 0.03
words = []
words_missing_in_tfidf = []


In [ ]:
print(texts[0][0:20])

## Vizualizing the data

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word=dictionary,
                                           num_topics=30,
                                           random_state = 100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=15,
                                           alpha="auto")

In [13]:
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds="mmds", R=30)
vis

/Users/mariemccormick/AciduleApp/lib/python3.11/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.196757  0.073809       1        1  26.160983
3     -0.184906  0.140369       2        1  17.039019
11    -0.289680  0.185599       3        1   9.083544
12    -0.253489  0.008604       4        1   7.506478
9     -0.213894  0.212908       5        1   6.170449
25    -0.380643 -0.026140       6        1   5.301554
13    -0.310462  0.060689       7        1   4.838648
8     -0.264903  0.102591       8        1   4.606181
0     -0.208490  0.274160       9        1   3.270054
23    -0.072827  0.264066      10        1   2.782606
18    -0.106170  0.210758      11        1   2.443913
24     0.064052  0.362800      12        1   2.043998
5      0.103867  0.300421      13        1   1.841953
28     0.365854  0.151145      14        1   1.084610
26     0.448962  0.190976      15        1   1.035972
15    -0.222151 -0.222712      16        1   0.887142
14    -0.073928  0.012944      17        1   0.838112
27    -0.165822 -0.137036      18        1   0.828564
2      0.488844  0.022036      19        1   0.813248
16    -0.130244 -0.312482      20        1   0.349276
17     0.233378 -0.327603      21        1   0.292291
6      0.298566 -0.171100      22        1   0.238278
1      0.202703  0.033493      23        1   0.220991
21     0.127216 -0.330487      24        1   0.192929
4      0.063555 -0.130802      25        1   0.094869
10     0.110941 -0.175691      26        1   0.033414
29     0.141608 -0.193329      27        1   0.000233
22     0.141608 -0.193329      28        1   0.000231
20     0.141608 -0.193329      29        1   0.000231
19     0.141608 -0.193329      30        1   0.000228, topic_info=            Term         Freq        Total Category  logprob  loglift
5223         you  1971.000000  1971.000000  Default  30.0000  30.0000
671          oui  2782.000000  2782.000000  Default  29.0000  29.0000
659          non  3401.000000  3401.000000  Default  28.0000  28.0000
485         bien  6767.000000  6767.000000  Default  27.0000  27.0000
646     monsieur  7479.000000  7479.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
695        petit     0.000064  3037.096313  Topic30 -10.1948  -4.6789
715      prendre     0.000064  2287.618058  Topic30 -10.1948  -4.3955
460        année     0.000064  2199.609068  Topic30 -10.1948  -4.3563
671          oui     0.000064  2782.086499  Topic30 -10.1948  -4.5912
1021  discussion     0.000064  1358.075016  Topic30 -10.1948  -3.8741

[2116 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
1556       3  0.253474     aacidule
1556       4  0.190105     aacidule
1556       5  0.190105     aacidule
1556       7  0.253474     aacidule
16737      8  0.930084       aborde
...      ...       ...          ...
2807      17  0.003432        étude
2807      18  0.017158        étude
2807      25  0.001144        étude
17907      5  0.940829  évangélique
7307       6  0.928916         ôter

[6178 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 4, 12, 13, 10, 26, 14, 9, 1, 24, 19, 25, 6, 29, 27, 16, 15, 28, 3, 17, 18, 7, 2, 22, 5, 11, 30, 23, 21, 20])